## Ambiente "local" de pruebas
En este notebook se hará el desarrollo del ambiente local de pruebas, en el que se incluirán las funciones necesarias y poder hacer un script final

In [1]:
import numpy as np
import pandas as pd
import time

from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('data/clean/train_clean.csv')
df_targets = pd.read_csv('data/clean/train_labels.csv')

### Pruebas

In [186]:
x_train = df[df['fecha_dato'] == '2015-01-28']
y_train = df_targets.loc[x_train.index]

In [13]:
x = x_train.drop(['fecha_dato', 'fecha_alta'], axis=1).as_matrix()
y = y_train.as_matrix()

In [187]:
x_test = df[df['fecha_dato'] == '2015-02-28']
y_test = df_targets.loc[x_test.index]

In [16]:
x_test = x_test.drop(['fecha_dato', 'fecha_alta'], axis=1).as_matrix()

In [14]:
rf = RandomForestClassifier(n_jobs=4)
rf = model(x, y, rf)

In [145]:
probs = rf.predict_proba(x_test)
preds = rf.predict(x_test)

In [146]:
probs = pred_probs = np.array([pr.max(axis=1) for pr in probs]).T
probs.shape

(516199, 24)

In [156]:
x_train.head()

,level_0,index,fecha_dato,fecha_dato_year,fecha_dato_month,fecha_dato_day,ncodpers,ind_empleado,pais_residencia,sexo,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,0,0,2015-01-28,2015,1,28,1375586,1,1,1,...,2,1,20,1,1.0,29.0,7,1.0,87218.10,1
1,1,1,2015-01-28,2015,1,28,1050611,1,1,1,...,2,2,1,1,1.0,13.0,23,0.0,35548.74,2
2,2,2,2015-01-28,2015,1,28,1050612,1,1,1,...,2,1,1,1,1.0,13.0,23,0.0,122179.11,2
3,3,3,2015-01-28,2015,1,28,1050613,1,1,1,...,2,1,7,1,1.0,50.0,6,0.0,119775.54,2
4,4,4,2015-01-28,2015,1,28,1050615,1,1,1,...,2,1,1,1,1.0,45.0,16,0.0,22220.04,2


In [222]:
%%time
predicted, actual = processPredictions(probs, preds, x_train, x_test, y_train, y_test)

CPU times: user 16.4 s, sys: 356 ms, total: 16.8 s
Wall time: 16.8 s


In [223]:
%%time
subm = processPredictions(probs, preds, x_train, x_test, y_train, y_test, env='submit')

CPU times: user 9.91 s, sys: 236 ms, total: 10.1 s
Wall time: 10.8 s


In [233]:
n = 2
apk(actual[n], predicted[n])

1.0

In [228]:
predicted[:10]

array([[ 2, 23, 22,  1,  3,  4,  5],
       [18, 23, 10,  1,  2,  3,  4],
       [ 2, 23, 22,  1,  3,  4,  5],
       [ 2, 23,  8, 22,  1,  3,  4],
       [23, 22,  1,  2,  3,  4,  5],
       [ 2, 23, 22,  1,  3,  4,  5],
       [23, 21,  4, 22, 18,  7,  9],
       [23,  2, 22,  1,  3,  4,  5],
       [ 2, 23, 22,  1,  3,  4,  5],
       [ 2, 23, 22,  1,  3,  4,  5]])

In [229]:
actual[:10]

[[2.0],
 [18.0],
 [2.0],
 [2.0, 8.0],
 [14.0, 15.0, 18.0, 21.0, 22.0, 23.0],
 [2.0],
 [4.0, 7.0, 18.0, 21.0, 22.0, 23.0],
 [2.0],
 [2.0, 17.0, 23.0],
 [2.0]]

### Entrenamiento del modelo

In [30]:
def model(x_train, y_train, model):
    """
    Parameters
    ----------
    x_train: Array
        Datos de entrenamiento previamente procesados
    y_train: Array
        Targets de entrenamiento
    model: Objeto (de sklearn)
        Algoritmo para entrenar con los paramétros configurados
    
    Returns
    -------
    model: Objeto
        Algortimo entrenado
    """
    return model.fit(x_train, y_train)

### Predicciones

In [221]:
def processPredictions(probs=None, preds=None, df_train=None, df_test=None,
                       df_targets=None, y_test=None, env='local', path='results/submissions/'):
    """
    Procesamiento de las predicciones hechas para generar el archivo de submission
    o los datos necesarios para hacer una validación local
    Parameters
    ----------
    probs: Array
        Probabilidades de elegir un producto - Generado por el modelo
    preds: Array
        Predicciones hechas por el modelo entrenado
    df_train: DataFrame
        Datos de entrenamiento - último mes de entrenamiento
    df_test: DataFrame
        Datos de test si se va a hacer una validación local, por el contrario es None por default
    df_targets: DataFrame
        Targets - último mes de entrenamiento
    y_test: DataFrame
        Targets del mes de test, sólo si env es 'local'
    env: str (optional)
        Indica el tipo de ejecución que se quiere hacer
            'local': regresa dos listas para hacer la validación local - Default
            'submit': regresa un DataFrame con el archivo de submission csv (el archivo de submission se genera
                    y se guarda en el equipo)
    path: str (optional)
        Dirección de la carpeta donde se va a guardar el archivo
    
    Returns
    -------
    Si env es 'submit'
        df_subm: DataFrame
            DataFrame con el archivo de submission csv (el archivo de submission se genera y se guarda en el equipo)
    Si env es 'local'
        predicted: list
            Una lista de listas de los productos que se predijeron
    
    """
    
    df_train.reset_index(drop=True, inplace=True)
    df_test.reset_index(drop=True, inplace=True)
    df_targets.reset_index(drop=True, inplace=True)
    
    ncodpers_prev_month = df_train.loc[:, 'ncodpers'].values
    ncodpers_last_month = df_test.loc[:, 'ncodpers'].values
    
    ncodpers_both = list(set(ncodpers_last_month) & set(ncodpers_prev_month))
    
    index_prev = df_train[df_train['ncodpers'].isin(ncodpers_both)].sort_values(['ncodpers']).index
    index_last = df_test[df_test['ncodpers'].isin(ncodpers_both)].sort_values(['ncodpers']).index
    
    prev_prods = df_targets.loc[index_prev].as_matrix()
    pred_prods = preds[index_last, :]
    
    both_prods = pred_prods - prev_prods
    both_prods = (both_prods < 0) * 1
    
    preds[index_last] = both_prods
    
    pred_probs = (preds * probs).argsort(axis=1)
    pred_probs = np.fliplr(pred_probs)[:, :7]
    
    if env == 'submit':
        targets = np.array(df_targets.columns.tolist())
        final_pred = [" ".join(list(targets[p])) for p in pred_probs]

        df_subm = pd.DataFrame({'ncodpers': df_test.ncodpers.values, 'added_products': final_pred})
        name_file = path + time.strftime("%Y-%m-%d-h%H-%M-%S_") + "submission.csv"
        df_subm.to_csv(name_file, index=False)
    
        return df_subm
    else:
        y = y_test.as_matrix()
        indexs = np.array([[i for i in range(y.shape[1])] * y.shape[0]]).reshape(y.shape[0], y.shape[1])
        actual = y * indexs
        actual = list(map(lambda x: list(np.unique(x)[1:]), actual))
        predicted = pred_probs
        
        return predicted, actual

### Métrica de desempeño
Para hacer mediciones locales

In [144]:
"""
From github: @benhamner
"""

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])